In [1]:
import matplotlib.gridspec as gridspec
import scqubits
import numpy as np
from CoupledQuantumSystems.visualization import *
from CoupledQuantumSystems.systems import *
from tqdm import tqdm
from CoupledQuantumSystems.qobj_manip import *
from CoupledQuantumSystems.drive import *
from scipy.optimize import minimize

In [12]:
Ej_list = np.linspace(34.35,34.45,50)
amp_list  = np.linspace(0.0075,0.008, 200)

In [13]:
me0001 = []
mel0l1 = []
me2021 = []
half_leak_rabi_period = []
detuned_zero_rabi_period = []
detuned_two_rabi_period = []
qbt_frequency_tmon0 = []
qbt_frequency_tmon1 = []

# max_overlap_for_q4t1 = []
tmon_freq_qbt_0 = []
tmon_freq_qbt_3 = []
tmon_freq_qbt_2 = []

EJ = 4
EC = 4/2
EL = 4/30


fluxonium = scqubits.Fluxonium(EJ=EJ,
                        EC=EC,
                        EL=EL,
                        flux=0,cutoff=110,
                        truncated_dim=20)
for Ej_transmon in tqdm(Ej_list):
    transmon = scqubits.Transmon(
        EJ=Ej_transmon,
        EC=0.2,
        ng=0.0,
        ncut=10,
        truncated_dim = 4
        )
    system = FluxoniumTransmonSystem(
        fluxonium  = fluxonium,
        transmon = transmon,
        computaional_states = '0,2',
        g_strength = 0.1,
        )
    
    energies, evecs = system.hilbertspace.hamiltonian().eigenstates()
    # overlap_matrix = scqubits.utils.spectrum_utils.convert_evecs_to_ndarray(evecs)
    # product_index = 1 * system.transmon.truncated_dim + 1
    # overlaps_with_product_state = np.abs(overlap_matrix[:, product_index])
    # sorted_indices = np.argsort(overlaps_with_product_state)[::-1]
    # sorted_overlaps = overlaps_with_product_state[sorted_indices]
    # max_overlap_for_q4t1.append(list(sorted_overlaps)[0]**2)

    def dressed_ener(ql,ol):
        return energies[system.product_to_dressed[(ql,ol)]]
    ols = system.transmon.truncated_dim
    op = system.hilbertspace.op_in_dressed_eigenbasis(transmon.n_operator)

    me0001.append(abs(op[system.product_to_dressed[(0,0)],  system.product_to_dressed[(0,1)]]))
    mel0l1.append(abs(op[system.product_to_dressed[(3,0)],  system.product_to_dressed[(3,1)]]))
    me2021.append(abs(op[system.product_to_dressed[(2,0)],  system.product_to_dressed[(2,1)]]))

    half_leak_rabi_period.append([])
    detuned_zero_rabi_period.append([])
    detuned_two_rabi_period.append([])
    
    for amp in amp_list:
        w_d = dressed_ener(3,1)-dressed_ener(3,0)
        half_leak_rabi_period[-1].append( np.pi/ (amp *abs(op[system.product_to_dressed[(3,0)],  system.product_to_dressed[(3,1)]])  ))
        detuned_zero_rabi_period[-1].append(
            2*np.pi / np.sqrt( 
                (amp * abs(op[system.product_to_dressed[(0,0)],  system.product_to_dressed[(0,1)]]))**2 +\
            (2*np.pi*(dressed_ener(0,1)-dressed_ener(0,0)) - 2*np.pi*w_d)**2
            )
        )
        detuned_two_rabi_period[-1].append(
            2*np.pi / np.sqrt( 
                (amp *abs(op[system.product_to_dressed[(2,0)],  system.product_to_dressed[(2,1)]]))**2 +\
            (2*np.pi*(dressed_ener(2,1)-dressed_ener(2,0)) - 2*np.pi*w_d)**2)
        )

    qbt_frequency_tmon0.append(2*np.pi*(dressed_ener(2,0)-dressed_ener(0,0)))
    qbt_frequency_tmon1.append(2*np.pi*(dressed_ener(2,1)-dressed_ener(0,1)))
    tmon_freq_qbt_0.append(2*np.pi*(dressed_ener(0,1)-dressed_ener(0,0)))
    tmon_freq_qbt_3.append(2*np.pi*(dressed_ener(3,1)-dressed_ener(3,0)))
    tmon_freq_qbt_2.append(2*np.pi*(dressed_ener(2,1)-dressed_ener(2,0)))

100%|██████████| 50/50 [03:21<00:00,  4.03s/it]


In [14]:
half_leak_rabi_period = np.array(half_leak_rabi_period)
detuned_zero_rabi_period = np.array(detuned_zero_rabi_period)
detuned_two_rabi_period = np.array(detuned_two_rabi_period)
qbt_frequency_tmon0 = np.array(qbt_frequency_tmon0)
qbt_frequency_tmon1 = np.array(qbt_frequency_tmon1)

In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Surface(z=detuned_zero_rabi_period,x=amp_list,y = Ej_list, colorscale = ['red','red'],opacity=1,showscale= False,name="Detuned One Rabi"),
    go.Surface(z=detuned_two_rabi_period,x=amp_list,y = Ej_list, colorscale = ['darkorange','darkorange'],opacity=1,showscale= False,name="Detuned Two Rabi"),
    go.Surface(z=half_leak_rabi_period,x=amp_list,y = Ej_list,colorscale = ['green','green'], opacity=0.8,showscale= False,name="Half Leak Rabi"),

    # go.Surface(z=detuned_zero_rabi_period * 2,x=amp_list,y = Ej_list, colorscale = ['red','red'],opacity=1,showscale= False,name="Detuned One Rabi"),
    # go.Surface(z=detuned_two_rabi_period * 2,x=amp_list,y = Ej_list, colorscale = ['darkorange','darkorange'],opacity=1,showscale= False,name="Detuned Two Rabi"),

    # go.Surface(z=detuned_zero_rabi_period * 3,x=amp_list,y = Ej_list, colorscale = ['red','red'],opacity=1,showscale= False,name="Detuned One Rabi"),
    # go.Surface(z=detuned_two_rabi_period * 3,x=amp_list,y = Ej_list, colorscale = ['darkorange','darkorange'],opacity=1,showscale= False,name="Detuned Two Rabi"),
])


# Adding dummy scatter3d traces for the legend
fig.add_trace(go.Scatter3d(x=[None], y=[None], z=[None], mode='markers',
                           marker=dict(size=10, color='green'),
                           legendgroup='group1', showlegend=True, name='Half Leak Rabi'))
fig.add_trace(go.Scatter3d(x=[None], y=[None], z=[None], mode='markers',
                           marker=dict(size=10, color='red'),
                           legendgroup='group2', showlegend=True, name='Detuned One Rabi'))
fig.add_trace(go.Scatter3d(x=[None], y=[None], z=[None], mode='markers',
                           marker=dict(size=10, color='darkorange'),
                           legendgroup='group3', showlegend=True, name='Detuned Two Rabi'))


fig.update_layout(scene=dict(
    xaxis_title='drive amplitude',  # Update this to your actual x-axis label
    yaxis_title='transmon flux',  # Update this to your actual y-axis label
    zaxis_title='t (ns)'   # Update this to your actual z-axis label
))
fig.update_layout(width=1000, height=1000)
fig.show()